# Practical session 4 - K-nearest neighbours (K-NN) classification with numpy, scikit-learn, cython and numba

Students (pair):
- [BENICHOU Yaniv](https://github.com/benech17)
- [BONNEFOY Nicolas](https://github.com/Nicolas-Bonnefoy)

**Useful references for this lab**:

[1] scikit-learn: [documentation](https://scikit-learn.org/stable/modules/neighbors.html?highlight=knn%20classification)

[2] `numba`: [documentation](http://numba.pydata.org/) 

[3] cython: [a very useful tutorial](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial), and [another one](http://docs.cython.org/en/latest/src/tutorial/cython_tutorial.html)



## <a name="content">Contents</a>
- [Exercise 1: KNN classification with numpy and sklearn](#ex1)
- [Exercise 2: Code acceleration with cython](#ex2)
- [Exercise 3: Code acceleration with numba](#ex3)
---

In [1]:
%load_ext autoreload
%autoreload 2

## <a name="ex1">Exercise 1: K-Nearest Neighbours (K-NN) classification with numpy and scikit-learn</a> [(&#8593;)](#content)

This session is a first introduction to classification using the most intuitive non parametric method: the $K$-nearest neighbours. The principle is [the following](https://scikit-learn.org/stable/modules/neighbors.html?highlight=knn%20classification). A set of labelled observations is given as a learning set. A classification taks then consists in assigning a label to any new observation. In particular, the K-NN approach consists in assigning to the observation the most frequent label among its $K$ nearest neighbours taken in the training set.

### A. Validation on synthetic data

Load the training and test datasets `data/synth_train.txt` and `data/synth_test.txt`. Targets belong to the set $\{1,2\}$ and entries belong to $\mathbb{R}^2$. The file `data/synth_train.txt` contain 100 training data samples, and `data/synth_test.txt` contains 200 test samples, where:

- the 1st column contains the label of the class the sample;
- columns 2 & 3 contain the coordinates of each sample (in $\mathbb{R}^2$).

Useful commands can be found below.

```python
# load the training set
train = np.loadtxt('data/synth_train.txt')  #...,delimiter=',') if there are ',' as delimiters
class_train = train[:,0]
x_train = train[:,1:]
N_train = train.shape[0]
```

```python
# load the test set
test = np.loadtxt('/datasynth_test.txt') 
class_test_1 = test[test[:,0]==1]
class_test_2 = test[test[:,0]==2]
x_test = test[:,1:]
N_test = test.shape[0]
```

1\. Display the training set and distinguish the two classes. 

> Hint: useful functions include `matplotlib.pyplot.scatter` or `matplotlib.pyplot.plot`.

**Answer:**

In [2]:
import numpy as np
import plotly.express as px

train = np.loadtxt('data/synth_train.txt')

# Séparation des labels et des coordonnées
class_train = train[:, 0].astype(int)
x_train = train[:, 1:]
N_train = train.shape[0]

test = np.loadtxt('data/synth_test.txt') 
class_test = test[:, 0].astype(int)
class_test_1 = test[test[:,0]==1]
class_test_2 = test[test[:,0]==2]
x_test = test[:,1:]
N_test = test.shape[0]


# Création du scatter plot avec plotly
fig = px.scatter(x=x_train[:, 0], y=x_train[:, 1], color=class_train.astype(str), 
                 labels={'color': 'Label', 'x': 'Coordonnée X', 'y': 'Coordonnée Y'},
                 title='Scatter plot des observations du Training Set')

fig.show()

2\. Implement the K-nearest neighbours algorithm for classification.

> Hint: 
> - useful functions include `numpy.linalg.norm`, `numpy.argsort`, `numpy.bincount`;
> - implement the algorithm as a function rather than an object. This will drastically simplify the acceleration step using Cython.
> - for an optimized partial sorting procedure, you may have a look at the [`bottleneck.argpartition` function](https://bottleneck.readthedocs.io/en/latest/reference.html#bottleneck.argpartition).
> 1. Compute for each row in `x_test` (if necessary use `np.newaxis`) its distance with respect to `x_train`:
>  - Use  `numpy.linalg.norm` (in which dimension this distance is computed ? Consider using `axis` argument)
> 2. Sort the ordered collection of distances (indices from smallest to largest (in ascending order) by the distances):
>   - Use `np.argsort` (at the end replace this procedure by `bottleneck.argpartition`)
>   - Once the sorting is done, we take only the indices of `labels` of the `n_neighbours` nearest neighbours of the `class_train` :
>     - `id = np.argsort(distances)[:n_ neighbours]` and `labels = class_train[id]`
> 3. The K-nearest can be used for **Regression**, in this case it is necessary to return the mean of the K-labels. For **Classification**,  we return the mode of the K-labels :
> - Use `np.bincount` for `labels` to affect the variable `class_pred[q]` (for row `q`). This procedure counts the number of occurrences of each value in array. **Mode** is the value that appears. How can we get this value ?


**Answer:**

In [3]:
import bottleneck

def knn_classification(x_train, class_train, x_test, n_neighbours=3):
    """
    K-nearest neighbours algorithm for classification.

    Parameters:
    - x_train: Training data
    - class_train: Class labels for training data
    - x_test: Test data
    - n_neighbours: Number of neighbours to consider

    Returns:
    - class_pred: Predicted class labels for test data
    """
    
    class_pred = np.empty(x_test.shape[0], dtype=class_train.dtype)
    
    for row, x in enumerate(x_test):
        distances = np.linalg.norm(x_train - x[np.newaxis, :], axis=1)  
        index = bottleneck.argpartition(distances, n_neighbours)[:n_neighbours]
        class_pred[row] = np.bincount(class_train[index]).argmax()

    return class_pred

3\. Compute the error rate on the training set and the test set for $K \in \{1,2, \dotsc, 20\}$. Display the classification result (see 1.) for the configuration with the lowest error rate.

**Answer:**

In [4]:
def compute_error_rate(true_labels, predicted_labels):
    return np.mean(true_labels != predicted_labels)

train_errors = []
test_errors = []

for K in range(1, 21):
    train_pred = knn_classification(x_train, class_train, x_train, n_neighbours=K)
    test_pred = knn_classification(x_train, class_train, x_test, n_neighbours=K)
    
    train_error = compute_error_rate(class_train, train_pred)
    test_error = compute_error_rate(class_test, test_pred)
    
    train_errors.append(train_error)
    test_errors.append(test_error)

best_K = np.argmin(test_errors) + 1  # +1 because K starts from 1, not 0 as usual

print(f"Best K: {best_K}")
print(f"Train Error Rate for Best K: {train_errors[best_K-1]}")
print(f"Test Error Rate for Best K: {test_errors[best_K-1]}")

# Result for best K
best_test_pred = knn_classification(x_train, class_train, x_test, n_neighbours=best_K)
print(f"Classification results for best K ({best_K}):")
print(best_test_pred)

Best K: 3
Train Error Rate for Best K: 0.03
Test Error Rate for Best K: 0.045
Classification results for best K (3):
[2 2 2 2 2 2 2 2 2 1 2 1 1 2 1 2 2 1 1 2 2 1 2 1 2 2 1 2 2 1 2 1 2 1 2 2 1
 1 2 2 1 2 2 2 2 2 2 1 1 2 2 1 1 1 2 2 1 2 2 2 2 2 2 1 2 2 2 1 2 2 2 1 2 2
 1 1 2 2 1 1 1 2 1 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2
 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 1 1 1 2 2 2
 1 2 2 2 2 2 1 2 2 2 2 2 1 2 2 1 2 2 2 1 2 2 1 2 2 2 1 2 2 1 2 2 2 2 2 2 1
 2 2 1 1 2 1 1 1 2 2 2 2 2 2 1]


4\. Comment on your results. Which value of $K$ seems optimal ?


**Answer:**

> The value of K which seems optimal is k=3 with a test error rate of 0.045. 

5\. Compare the results of you implementation with those of [`sklearn.neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html?highlight=kneighborsclassifier#sklearn.neighbors.KNeighborsClassifier). Compare the runtime of these two versions using the [`timeit`](https://docs.python.org/3/library/timeit.html) module (see session 1).

**Answer:**

In [5]:

from sklearn.neighbors import KNeighborsClassifier
import timeit
import plotly.graph_objects as go

def sklearn_knn(x_train, class_train, x_test, K):
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(x_train, class_train)
    return knn.predict(x_test)

def knn_maison(x_train, class_train, x_test, K):
    return knn_classification(x_train, class_train, x_test, n_neighbours=K)

# Lists to stock run times
our_times = []
sklearn_times = []

# Compare results and runtime
for K in range(1, 21):
    #our_pred = knn_maison(x_train, class_train, x_test, K)
    sklearn_pred = sklearn_knn(x_train, class_train, x_test, K)
        
    #our_time = timeit.timeit(lambda: knn_maison(x_train, class_train, x_test, K), number=10)
    sklearn_time = timeit.timeit(lambda: sklearn_knn(x_train, class_train, x_test, K), number=10)
    
   # our_times.append(our_time)
    sklearn_times.append(sklearn_time)

    print(f"Pour K={K}:  sklearn KNN : {sklearn_time:.4f}s")

for K in range(1, 21):
    our_pred = knn_maison(x_train, class_train, x_test, K)
    sklearn_pred = sklearn_knn(x_train, class_train, x_test, K)
        
    our_time = timeit.timeit(lambda: knn_maison(x_train, class_train, x_test, K), number=10)
    sklearn_time = timeit.timeit(lambda: sklearn_knn(x_train, class_train, x_test, K), number=10)
    
    our_times.append(our_time)
    sklearn_times.append(sklearn_time)

    print(f"Pour K={K}: Notre KNN : {our_time:.4f}s ")


K_values = list(range(1, 21))

# Affichage
fig = go.Figure(data=[
    go.Bar(name='Notre KNN', x=K_values, y=our_times),
    go.Bar(name="sklearn's KNN", x=K_values, y=sklearn_times)
])
fig.update_layout(barmode='group', title_text='Comparaison des temps d\'exécution de KNN',
                  xaxis_title='Valeur de K', yaxis_title='Temps d\'exécution (s)')
fig.show();

Pour K=1:  sklearn KNN : 0.0266s
Pour K=2:  sklearn KNN : 0.0269s
Pour K=3:  sklearn KNN : 0.0275s
Pour K=4:  sklearn KNN : 0.0274s
Pour K=5:  sklearn KNN : 0.0276s
Pour K=6:  sklearn KNN : 0.0278s
Pour K=7:  sklearn KNN : 0.0286s
Pour K=8:  sklearn KNN : 0.0289s
Pour K=9:  sklearn KNN : 0.0284s
Pour K=10:  sklearn KNN : 0.0284s
Pour K=11:  sklearn KNN : 0.0290s
Pour K=12:  sklearn KNN : 0.0289s
Pour K=13:  sklearn KNN : 0.0290s
Pour K=14:  sklearn KNN : 0.0300s
Pour K=15:  sklearn KNN : 0.0315s
Pour K=16:  sklearn KNN : 0.0310s
Pour K=17:  sklearn KNN : 0.0307s
Pour K=18:  sklearn KNN : 0.0307s
Pour K=19:  sklearn KNN : 0.0303s
Pour K=20:  sklearn KNN : 0.0307s
Pour K=1: Notre KNN : 0.0197s 
Pour K=2: Notre KNN : 0.0195s 
Pour K=3: Notre KNN : 0.0196s 
Pour K=4: Notre KNN : 0.0197s 
Pour K=5: Notre KNN : 0.0196s 
Pour K=6: Notre KNN : 0.0197s 
Pour K=7: Notre KNN : 0.0197s 
Pour K=8: Notre KNN : 0.0197s 
Pour K=9: Notre KNN : 0.0198s 
Pour K=10: Notre KNN : 0.0201s 
Pour K=11: Notre K

In [6]:
# Graphique version lineplot
fig = go.Figure()
fig.add_trace(go.Scatter(name='Notre KNN', x=K_values, y=our_times, mode='lines+markers'))
fig.add_trace(go.Scatter(name="sklearn's KNN", x=K_values, y=sklearn_times, mode='lines+markers'))
fig.update_layout(barmode='group', title_text='Comparaison des temps d\'exécution de KNN',
                  xaxis_title='Valeur de K', yaxis_title='Temps d\'exécution (s)')
fig.show();

### B. Application to a real dataset (Breast cancer Wisconsin).

6\. Apply the K-NN classifier to the real dataset `data/wdbc12.data.txt.` Further details about the data are provided in `data/wdbc12.names.txt`.

> Hint: you can use the function [`train_test_split` from `sklearn.model_selection`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the dataset into a training and a test set.

**Answer:**

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 

df = np.loadtxt('data/wdbc12.data.txt',delimiter=',')
y = df[:,1].astype(int)
x = df[:,2:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

def testWithTrainOnTest(x_train,x_test,y_train,K=3):
    y_pred=np.empty(x_test.shape[0],dtype=y_train.dtype)
    for row,x in enumerate(x_test):
        distances=np.linalg.norm(x_train-x[np.newaxis,:],axis=1)
        index=bottleneck.argpartition(distances,K)[:K]
        y_pred[row]=np.bincount(y_train[index]).argmax()
    return y_pred

target_prediction_test=testWithTrainOnTest(x_train,x_test,y_train,K=3)

def computeError(target_prediction,y):
    cm=confusion_matrix(target_prediction,y)
    return cm

error=computeError(target_prediction_test,y_test)
print(error)
error_rate=(error[0,1]+error[1,0])/error.sum()
print('error rate is' ,round(100*error_rate,2),'%')

[[49  5]
 [ 5 84]]
error rate is 6.99 %


## <a name="ex2">Exercise 2: Code acceleration with cython</a> [(&#8593;)](#content)

Cython allows C code to be easily interfaced with Python. It can be useful to make your code faster for a small coding effort, in particular when using loops. A general approach to optimize your code is outlined in the [Scipy lecture notes, Section 2.4](https://scipy-lectures.org/advanced/optimizing/index.html). Complementary reading about interfacing Python with C can be found in [Section 2.8](https://scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html).

1\. Read carefully the [cython tutorial](http://docs.cython.org/en/latest/src/tutorial/cython_tutorial.html), which describes step by the step how the toy example reported below has been developed.

**Setup**: Compile the toy example provided in `example_cy/` by running, in the command line (anaconda prompt on windows)

```bash
cd example_cy && python setup.py build_ext --inplace
```

Note that the compilation process has been slightly automatised with the instructions reported in `example_cy/setup.py`. To test the module, run

In [26]:
!cd example_cy && python setup.py build_ext --inplace

running build_ext
copying build/lib.macosx-11.1-arm64-cpython-39/example_cy/helloworld.cpython-39-darwin.so -> example_cy
copying build/lib.macosx-11.1-arm64-cpython-39/example_cy/primes.cpython-39-darwin.so -> example_cy


In [27]:
import example_cy.example_cy.helloworld as toy

toy.printhello()

Hello World


which should display
```python
Hello World
```

> Warning: 
> - do not forget to include an empty `__init__.py` file in the directory where your source code lives (`import` will fail if this is not the case).
> - in case you have any setup issue, take a look at the `notes.md` file.
> - if the C code and/or the executable do not seem to be regenerated by the build instructions, delete the C code and the executable first, and re-execute the compilation afterwards.
> - do not hesitate to restart the Python kernel if necessary when the Cython executable has been re-generated.

2\. Read the [Numpy/Cython tutorial](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial), focussing on the paragraphs **Cython at a glance**, and **Your Cython environment** until **"More generic code"**. An example to compile a `.pyx` file depending on `numpy` is included in `example_np_cy/`.

> Remarks: 
> - the `annotate=True` flag in the `setup.py` allows an additional `.html` document to be generated (`<your_module_name>.html`), showing, for each line of the Cython code, the associated C instructions generated. Highlighted in yellow are the interactions with Python: the darker a region appears, the less efficient the generated C code is for this section. Work in priority on these! 
> - make sure all the previously generated files are deleted to allow the .html report to be generated;
> - if you are working on your own machine and don't have a C/C++ compiler installed, read the notes provided in `notes.md`;
> - use `cdef` for pure C functions (not exported to Python), `cpdef` should be favored for functions containing C instructions and later called from Python.

**Answer:**

In [ ]:
# your code

3\. Use Cython to implement a faster version of the numpy K-NN classifier implemented in [Exercise 1](#ex1). To do so, apply step-by-step the techniques introduced in the [Numpy/Cython tutorial](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html#numpy-tutorial) (*i.e.*, compile and time your code after each step to report the evolution, keeping track of the different versions of the cython function).

> Hint: if you keep numpy arrays, make sure you use memory views (see numpy/cython tutorial) to access the elements within it. Be extremely careful with the type of the input arrays (you may need to recast the format of the input elements before entering the function. The `numpy.asarray` function can prove useful).

> **Detailed guidelines**: a few notes and *caveat* to help you re-writing your code in cython:
> - try to reduce the number of calls to numpy instructions as much as possible;
> - **you do not have to optimize everything**. For the KNN function above, most of the time is spent in computing euclidean distances: you can thus focus on optimizing tihs operations by explicitly writing a for loop, which will ensure a minimal interaction with numpy when generating the associated C code at compilation. Calls to other numpy functions can be kept as-is;
> - if you need to create an array within the cython function, used np.zeros (**do NOT use python lists**), and use a memory view to access its content;
> - specify the type for all variables and numpy arrays. Pay attention to the type of the input arrays passed to the Cython function;
> - whenever an array is returned, use memory views and index(es) to efficiently access its content;
> - some numpy operators (e.g., broadcasting mechanism) do not work with memory views. In this case, you can directly write for loop(s) to encode the operation of interest (the loops will be optimized out at compile time);
> - only use at the final development stage the following cython optimization (not before, as they can crash the program without any help):
>
>```python
>@cython.boundscheck(False)
>@cython.wraparound(False)
>```

**Answer:**

4\. Compare the runtime of the two algorithms (using `timeit.timeit`), and conclude about the interest of using cython in this case.

**Answer:**

In [ ]:
# your code

## <a name="ex3">Exercise 3: Code acceleration with numba</a> [(&#8593;)](#content)

`numba` is a just-in-time (JIT) compiler which translates Python codes into efficient machine code at runtime. A significant acceleration can be obtained by adding a few simple decorators to a standard Python function, up to a few restrictions detailed [here](http://numba.pydata.org/numba-doc/latest/user/performance-tips.html).

If you have written most of the KNN classifier of exercise 1 with numpy, there is little to no chance that you will get an acceleration with numba (justifying the use of cython in this case). An interesting acceleration factor can however be obtained for the computation of the total variation investigated in session 2.

1\. Take a look at the [numba 5 min tour](http://numba.pydata.org/numba-doc/latest/user/5minguide.html), and accelerate the total variation code from session 2 with the `@jit` decorator. You may have to rewrite small portions of your code to get the expected acceleration (see [performance tips](http://numba.pydata.org/numba-doc/latest/user/performance-tips.html)).

**Answer:**

In [14]:
from numba import jit

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# lab2
def gradient2D(X):
    '''
    Computes the gradient of a 2D array
    
        Parameters:
            X: 2D array

        Returns:
            xdh: gradient along the x axis
            dvx: gradient along the y axis    
    '''
    
    assert X.ndim == 2, "X must be a 2D array"
    xdh=np.diff(X,axis=1)
    dvx=np.diff(X.T,axis=1)
    
    xdh=np.concatenate((xdh,np.zeros((X.shape[0],1))),axis=1)
    
    dvx=np.concatenate((dvx,np.zeros((X.shape[1],1))),axis=1).T

    return xdh,dvx

@jit
def gradient2D_numba(X):
    assert X.ndim == 2, "X must be a 2D array"
    
    # Compute the differences along the two dimensions
    xdh = np.diff(X, axis=1)
    dvx = np.diff(X.T, axis=1)
    
    # Append zeros to maintain the same shape as X
    xdh = np.concatenate((xdh, np.zeros((X.shape[0], 1))), axis=1)
    dvx = np.concatenate((dvx, np.zeros((X.shape[1], 1))), axis=1)
    
    return xdh, dvx.T

# lab2
def tv(X):
    '''
    Computes the total variation of a 2D array by using the gradient2D function
    
        Parameters:
            X: 2D array

        Returns:
            tv: total variation of X
    '''


    xdh,dvx=gradient2D(X)
    tv=0
    for m in range(X.shape[0]):
        for n in range(X.shape[1]):
            tv+=np.sqrt(xdh[m,n]**2+dvx[m,n]**2)
    return tv 
# np.sum 

@jit
def tv_numba(X:np.array)->float:
    """This function compute the discrete isotropic total variation of an input matrix in C^(M,N)

    Args:
        X (np.array): A matrix in C^(MxN)

    Returns:
        float: returns the value of the TV for the input matrix X
    """
    XDh, DvX = gradient2D_numba(X)
    sum = 0
    for m in range(XDh.shape[0]):
        for n in range(XDh.shape[1]):
            sum+= np.sqrt(XDh[m,n]**2 + DvX[m,n]**2)
    return sum


loop = 100
rng = np.random.default_rng(84548)
M,N = rng.integers(low=50, high=100, size = 2)
X = rng.random((M,N))
tv_numba(X)


<ipython-input-14-027e397254b5>:36: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

<ipython-input-14-027e397254b5>:71: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

<ipython-input-14-027e397254b5>:80: NumbaDeprecationWarning:


Fall-back from the nopython compilation path to the object mode compilation path has been detected. This is deprecated behaviour that wil

2502.4341657454056

In [13]:

# Test que tv = tv_numba

x=np.array([[1,2,3],[4,5,6],[7,8,9]])
print(tv(x)==tv_numba(x)) 

True


2\. Compare the runtime of the your numpy implementation and the `numba`-accelerated version (using `timeit.timeit`). 
> **Warning**: first run the numba version once to trigger the compilation, and then time it as usual. This is needed to avoid including the JIT compilation step in the runtime.

**Answer:**

In [11]:
loop = 100
rng = np.random.default_rng(84548)
M,N = rng.integers(low=50, high=100, size = 2)
X = rng.random((M,N))

execution_time = timeit.timeit(lambda: tv(X), number=loop) / loop

execution_time_numba = timeit.timeit(lambda: tv_numba(X), number=loop) / loop

print(execution_time/execution_time_numba)

261.41716727021867


> L'implémentation avec numba est 261 fois plus rapide ici, cela montre bien l'utilité de numba ici.